In [62]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import powerlaw
from scipy import stats
from scipy.stats import weibull_min, poisson
import os
from pathlib import Path
import seaborn as sns
from mpmath import zeta as mp_zeta

INSTITUICOES = ['uft', 'ufnt', 'ceulp', 'ifto', 'unitins', 'tocantins']

In [63]:
def zeta(s, k_min=1):
    """Função zeta de Riemann truncada para cálculo da Power Law discreta"""
    return float(mp_zeta(s, k_min))

def weibull_loglik(data, params):
    """Calcula o log-likelihood para distribuição Weibull"""
    return np.sum(weibull_min.logpdf(data, params['shape'], 0, params['scale']))

In [64]:
def compare_distributions(degree_values):
    """Realiza o ajuste e comparação de todas as distribuições"""
    fit = powerlaw.Fit(degree_values, discrete=True)
    shape, loc, scale = weibull_min.fit(degree_values, floc=0)
    weibull_params = {'shape': shape, 'scale': scale}
    mu = np.mean(degree_values)
    poisson_params = {'mu': mu}
    
    k_values = np.array(degree_values)
    k_min = fit.power_law.xmin
    n = len(k_values[k_values >= k_min])
    
    # 1. Cálculo para Power Law (Lei de Potência)
    alpha = fit.power_law.alpha
    loglik_powerlaw = -n * np.log(zeta(alpha, k_min)) - alpha * np.sum(np.log(k_values[k_values >= k_min]))
    
    # 2. Cálculo para Exponential
    Lambda = fit.exponential.Lambda
    loglik_exponential = n * np.log(Lambda) - Lambda * np.sum(k_values[k_values >= k_min])
    
    # 3. Cálculo para Lognormal
    mu_ln = fit.lognormal.mu
    sigma_ln = fit.lognormal.sigma
    loglik_lognormal = np.sum(-(np.log(k_values[k_values >= k_min]) - mu_ln)**2 / (2 * sigma_ln**2) - 
                       np.log(k_values[k_values >= k_min] * sigma_ln * np.sqrt(2 * np.pi)))
    
    # 4. Weibull
    loglik_weibull = weibull_loglik(k_values, weibull_params)
    
    # 5. Poisson
    loglik_poisson = poisson.logpmf(k_values, poisson_params['mu']).sum()
    
    models = {
        'Power Law': {
            'loglik': loglik_powerlaw,
            'params': 2,  # alpha e xmin
            'alpha': alpha,
            'xmin': k_min
        },
        'Exponential': {
            'loglik': loglik_exponential,
            'params': 1,  # Lambda
            'Lambda': Lambda
        },
        'Lognormal': {
            'loglik': loglik_lognormal,
            'params': 2,  # mu e sigma
            'mu': mu_ln,
            'sigma': sigma_ln
        },
        'Weibull': {
            'loglik': loglik_weibull,
            'params': 2,  # shape e scale
            'shape': weibull_params['shape'],
            'scale': weibull_params['scale']
        },
        'Poisson': {
            'loglik': loglik_poisson,
            'params': 1,  # mu
            'mu': poisson_params['mu']
        }
    }
    
    # Calcular AIC e pesos de Akaike
    results = []
    for name, model in models.items():
        aic = 2 * model['params'] - 2 * model['loglik']
        results.append({
            'Modelo': name,
            'Log-Likelihood': model['loglik'],
            'Parâmetros': model['params'],
            'AIC': aic,
            **{k: v for k, v in model.items() if k not in ['loglik', 'params']}
        })
    
    results_df = pd.DataFrame(results).sort_values('AIC')
    results_df['ΔAIC'] = results_df['AIC'] - results_df['AIC'].min()
    results_df['Peso AIC'] = np.exp(-0.5 * results_df['ΔAIC'])
    results_df['Peso AIC'] = results_df['Peso AIC'] / results_df['Peso AIC'].sum()
    
    return results_df, fit

In [65]:
def plot_individual_distributions_seaborn(institution, degree_values, save_dir, fit, weibull_params, poisson_params):
    """
    Plota cada distribuição ajustada em gráficos separados usando Seaborn,
    com fontes maiores e sem títulos.
    """
    sns.set_theme(
        style="whitegrid", 
        font='serif',
        rc={'font.serif': ['Times New Roman']}
    )
    sns.set_context("talk")

    # Preparação dos dados
    degree_counts = pd.Series(degree_values).value_counts().sort_index()
    P_k = degree_counts / len(degree_values)
    k_range = np.arange(min(degree_values), max(degree_values) + 1)

    output_path = Path(save_dir) / institution
    output_path.mkdir(parents=True, exist_ok=True)
    
    AXIS_LABEL_SIZE = 20
    LEGEND_SIZE = 18

    # --- 1. Gráfico da Lei de Potência ---
    plt.figure(figsize=(12, 8))
    ax = sns.scatterplot(x=P_k.index, y=P_k.values, s=100, label='Dados Empíricos')
    fit.power_law.plot_pdf(ax=ax, color='r', linestyle='--', 
                           label=f'Lei de Potência (α={fit.power_law.alpha:.2f})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Grau (k)', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Probabilidade P(k)', fontsize=AXIS_LABEL_SIZE)
    ax.legend(fontsize=LEGEND_SIZE)
    plt.savefig(output_path / f'{institution}_power_law.png', dpi=300, bbox_inches='tight')
    plt.close()

    # --- 2. Gráfico da Distribuição Exponencial ---
    plt.figure(figsize=(12, 8))
    ax = sns.scatterplot(x=P_k.index, y=P_k.values, s=100, label='Dados Empíricos')
    fit.exponential.plot_pdf(ax=ax, color='g', linestyle=':', 
                             label=f'Exponencial (λ={fit.exponential.Lambda:.2f})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Grau (k)', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Probabilidade P(k)', fontsize=AXIS_LABEL_SIZE)
    ax.legend(fontsize=LEGEND_SIZE)
    plt.savefig(output_path / f'{institution}_exponential.png', dpi=300, bbox_inches='tight')
    plt.close()

    # --- 3. Gráfico da Distribuição Lognormal ---
    plt.figure(figsize=(12, 8))
    ax = sns.scatterplot(x=P_k.index, y=P_k.values, s=100, label='Dados Empíricos')
    fit.lognormal.plot_pdf(ax=ax, color='b', linestyle='-.', 
                           label=f'Lognormal (μ={fit.lognormal.mu:.2f}, σ={fit.lognormal.sigma:.2f})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Grau (k)', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Probabilidade P(k)', fontsize=AXIS_LABEL_SIZE)
    ax.legend(fontsize=LEGEND_SIZE)
    plt.savefig(output_path / f'{institution}_lognormal.png', dpi=300, bbox_inches='tight')
    plt.close()

    # --- 4. Gráfico da Distribuição Weibull ---
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x=P_k.index, y=P_k.values, s=100, label='Dados Empíricos')
    weibull_fit = weibull_min.pdf(k_range, weibull_params['shape'], 0, weibull_params['scale'])
    ax = sns.lineplot(x=k_range, y=weibull_fit, color='m', linestyle='--', 
                      label=f'Weibull (λ={weibull_params["scale"]:.2f}, β={weibull_params["shape"]:.2f})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Grau (k)', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Probabilidade P(k)', fontsize=AXIS_LABEL_SIZE)
    ax.legend(fontsize=LEGEND_SIZE)
    plt.savefig(output_path / f'{institution}_weibull.png', dpi=300, bbox_inches='tight')
    plt.close()

    # --- 5. Gráfico da Distribuição Poisson ---
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x=P_k.index, y=P_k.values, s=100, label='Dados Empíricos')
    poisson_fit = poisson.pmf(k_range, poisson_params['mu'])
    ax = sns.lineplot(x=k_range, y=poisson_fit, color='c', linestyle='-', 
                      label=f'Poisson (μ={poisson_params["mu"]:.2f})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Grau (k)', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Probabilidade P(k)', fontsize=AXIS_LABEL_SIZE)
    ax.legend(fontsize=LEGEND_SIZE)
    plt.savefig(output_path / f'{institution}_poisson.png', dpi=300, bbox_inches='tight')
    plt.close()

In [67]:
def plot_degree_distribution(institution, save_dir=None):
    file_path = f'../results/metrics/{institution}/{institution}_2024.json'
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    degree_distribution = data['degree_distribution']
    
    df = pd.DataFrame(list(degree_distribution.items()), columns=['Grau', 'Frequência'])
    df['Grau'] = pd.to_numeric(df['Grau'])
    df = df.sort_values('Grau')

    df = df[df['Frequência'] > 0]
    
    sns.set_theme(
        style="whitegrid", 
        font='serif',
        rc={'font.serif': ['Times New Roman']}
    )
    sns.set_context("talk")
    plt.figure(figsize=(10, 6))
    
    ax = sns.histplot(x='Grau', weights='Frequência', data=df, 
                     discrete=True, stat='count', alpha=0.7)
    ax.set_yscale('log')
    ax.set_ylim(bottom=1)
    ax.tick_params(labelsize=12)
    ax.set_xlabel('Grau do Vértice', fontsize=14)
    ax.set_ylabel('Número de Vértices (Frequência)', fontsize=14)
    
    plt.xticks(rotation=0, ha='right')
    plt.tight_layout()
    
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_path = Path(f'{save_dir}/{institution}/') / f'{institution}_degree_distribution.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    # Extrair os valores de grau para os ajustes
    degrees = data['degrees']
    degree_values = [k for k in degrees.values() if k > 0]
    
    # Realizar a análise comparativa
    results_df, fit = compare_distributions(degree_values)
    
    # Plotar os gráficos individuais para cada distribuição
    if save_dir:
        shape, loc, scale = weibull_min.fit(degree_values, floc=0)
        weibull_params = {'shape': shape, 'scale': scale}
        mu = np.mean(degree_values)
        poisson_params = {'mu': mu}
        
        plot_individual_distributions_seaborn(institution, degree_values, save_dir, 
                                    fit, weibull_params, poisson_params)
    
    return results_df, data

In [ ]:
save_dir = "../results/img/distribuicoes/".strip()
analysis_results = {}

if save_dir and not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Processa cada instituição
for instituicao in INSTITUICOES:
    try:
        print(f"\nProcessando {instituicao.upper()}...")
        results_df, data = plot_degree_distribution(instituicao, save_dir)
        
        # Armazenar resultados
        analysis_results[instituicao] = {
            'model_comparison': results_df,
            'network_stats': {
                'num_nodes': data['num_nodes'],
                'num_edges': data['num_edges'],
                'avg_degree': np.mean([k for k in data['degrees'].values() if k > 0]),
                'max_degree': max(data['degrees'].values())
            }
        }
        
        # Imprimir resultados
        print(f"\nResultados para {instituicao.upper()}:")
        print("Comparação de Modelos:")
        print(results_df.to_markdown(tablefmt="grid", floatfmt=".2f"))
        
        print("\nEstatísticas da Rede:")
        print(f"- Número de nós: {data['num_nodes']}")
        print(f"- Número de arestas: {data['num_edges']}")
        print(f"- Grau médio: {np.mean([k for k in data['degrees'].values() if k > 0]):.2f}")
        print(f"- Grau máximo: {max(data['degrees'].values())}")
        
    except Exception as e:
        print(f"Erro ao processar {instituicao}: {str(e)}")

print("\nConcluído!")